In [1]:
!git clone -b chapter_13 https://github.com/maxpumperla/deep_learning_and_the_game_of_go.git

Cloning into 'deep_learning_and_the_game_of_go'...
remote: Enumerating objects: 33, done.
remote: Counting objects: 100% (33/33), done.
remote: Compressing objects: 100% (28/28), done.
remote: Total 894 (delta 9), reused 15 (delta 3), pack-reused 861
Receiving objects: 100% (894/894), 320.39 MiB | 41.57 MiB/s, done.
Resolving deltas: 100% (445/445), done.
Checking out files: 100% (133/133), done.


In [2]:
ls -la deep_learning_and_the_game_of_go/code/dlgo/data

total 52
drwxr-xr-x  2 root root 4096 Jan 21 03:18 ./
drwxr-xr-x 16 root root 4096 Jan 21 03:18 ../
-rw-r--r--  1 root root  114 Jan 21 03:18 __init__.py
-rw-r--r--  1 root root 1989 Jan 21 03:18 generator.py
-rw-r--r--  1 root root 4043 Jan 21 03:18 index_processor.py
-rw-r--r--  1 root root 8438 Jan 21 03:18 parallel_processor.py
-rw-r--r--  1 root root 9251 Jan 21 03:18 processor.py
-rw-r--r--  1 root root 5620 Jan 21 03:18 sampling.py


In [3]:
ls -la ../input/

total 20
drwxr-xr-x 5 root   root    4096 Jan 21 03:18 ./
drwxr-xr-x 6 root   root    4096 Jan 21 03:18 ../
drwxr-xr-x 4 nobody nogroup 4096 Jan 21 03:18 alphago-dlgo-create-h5py-oneplaneencoder/
drwxr-xr-x 4 nobody nogroup 4096 Jan 12 08:28 go-train-data/
drwxr-xr-x 2 nobody nogroup 4096 Jan 13 12:04 go-train-data-2/


In [4]:
ls -la ../input/alphago-dlgo-create-h5py-oneplaneencoder

total 1021408
drwxr-xr-x 4 nobody nogroup      4096 Jan 21 03:18 ./
drwxr-xr-x 5 root   root         4096 Jan 21 03:18 ../
-rw-r--r-- 1 nobody nogroup   4017526 Jan 21 03:18 __notebook__.ipynb
-rw-r--r-- 1 nobody nogroup       589 Jan 21 03:18 __output__.json
-rw-r--r-- 1 nobody nogroup   3525514 Jan 21 03:18 __results__.html
-rw-r--r-- 1 nobody nogroup         0 Jan 21 03:18 custom.css
drwxr-xr-x 2 nobody nogroup      4096 Jan 21 03:18 data/
drwxr-xr-x 5 nobody nogroup      4096 Jan 21 03:18 deep_learning_and_the_game_of_go/
-rw-r--r-- 1 nobody nogroup     42177 Jan 21 03:18 kgs_index.html
-rw-r--r-- 1 nobody nogroup   1617632 Jan 21 03:18 test_100_oneplane.hdf5
-rw-r--r-- 1 nobody nogroup      3595 Jan 21 03:18 test_samples.py
-rw-r--r-- 1 nobody nogroup 155493278 Jan 21 03:18 train_10000_oneplane.hdf5
-rw-r--r-- 1 nobody nogroup  15172666 Jan 21 03:18 train_1000_oneplane.hdf5
-rw-r--r-- 1 nobody nogroup   1435644 Jan 21 03:18 train_100_oneplane.hdf5
-rw-r--r-- 1 nobody nogroup 78706

In [5]:
import sys
sys.path.append('deep_learning_and_the_game_of_go/code')

In [6]:
mkdir checkpoints

In [7]:
import io
import zipfile, h5py
import re, os
import numpy as np

from dlgo.data.parallel_processor import GoDataProcessor
from dlgo.encoders.oneplane import OnePlaneEncoder as PlaneEncoder
#from dlgo.encoders.simple import SimpleEncoder as PlaneEncoder
from dlgo.networks import small, medium, large

from keras.models import Sequential
from keras.layers.core import Dense
from keras.callbacks import ModelCheckpoint
# from tensorflow.python.keras.models import Sequential
# from tensorflow.python.keras.layers.core import Dense
# from tensorflow.python.keras.callbacks import ModelCheckpoint

Using TensorFlow backend.


In [8]:
src_train_hdf5 = '../input/alphago-dlgo-create-h5py-oneplaneencoder/train_1000_oneplane.hdf5'
src_test_hdf5 = '../input/alphago-dlgo-create-h5py-oneplaneencoder/test_100_oneplane.hdf5'
with open(src_train_hdf5, 'rb') as f:
    hdf5_train_buf = io.BytesIO(f.read())
with open(src_test_hdf5, 'rb') as f:
    hdf5_test_buf = io.BytesIO(f.read())
hdf5_train_buf, hdf5_test_buf

(<_io.BytesIO at 0x7f0fbcadb8e0>, <_io.BytesIO at 0x7f0f788c9fc0>)

In [9]:
hdf5_train = h5py.File(hdf5_train_buf, "r")
hdf5_test = h5py.File(hdf5_test_buf, 'r')
hdf5_train, hdf5_test

(<HDF5 file "<_io.BytesIO object at 0x7f0fbcadb8e0>" (mode r)>,
 <HDF5 file "<_io.BytesIO object at 0x7f0f788c9fc0>" (mode r)>)

In [10]:
#print(hdf5_train['data'].keys())
print(hdf5_train['data/KGS-2015-19-8133-train_features_0'].shape)
print(hdf5_train['data/KGS-2015-19-8133-train_labels_0'].shape)
print(np.array(hdf5_train['data/KGS-2015-19-8133-train_labels_0']))

(256, 1, 19, 19)
(256,)
[ 60  73 307 269 268 249 287 231 263 317 174 290  71 327 326  52  51  53
  69 111  46 282 262 205 279 315 277 202 299 318 320 319 283 301 245 302
 187  97 117  40  59  41  61  58  77  39  78  43  42  23  62  44  63 186
 168 147  45  86 106  49  31  84 123  28  26  24  47  48  30 122 104 103
  85 159 161 172 173 191 153 197 199  89  70 119 211 192 193 210 141 140
 101 120 135 206 207 226 188 225 227 284 129 264 244 143 142 259 278 261
 242 260 258 222 280 281 220 241 130 112 246 145 110 181 180 212  72  92
  54  35  55  36 235 100 163 223 243 124 162 127 107 108  33  34 184 183
 164 165 167  91  90 109  74 131 150 148 149  93 166 185 146  66  65 165
 132  56 146 238 239 165 113  75 146 219 200 165 208 203 170  88 105  87
  29  60  53 307 269 299 306 325 287 310 324 313 318 317 319 298 263  43
 154  91  92  72  73  52 110  54  34  55  33  70  74 111 112 130 131 149
  69  89  51  75  93  50  71  90  88 128  49  31 107 127  36  30  48  29
  28  47  27  46  45  68  6

In [11]:
import itertools, random
from keras.utils import to_categorical

class Generator(object):
    
    def __init__(self, hdf5_file, data_type, nn=1):
        self.hdf5_file = hdf5_file
        self.data_type = data_type
        self.nn = nn
        self.get_file_list()
        self.num_samples = None
        self.get_num_samples()
    
    def get_file_list(self):
        name_list0 = list(self.hdf5_file.keys())
        #print(name_list0)
        name_list = [re.sub('features|labels', '{}', ee) for ee in name_list0]
        self.name_list = list(set(name_list))
    
    def get_num_samples(self, batch_size=128, num_classes=19 * 19):  # <2>
        if self.num_samples is not None:
            return self.num_samples
        else:
            self.num_samples = 0
            for x, y in self._generate(count_flag=True):
                #print(y)
                self.num_samples += len(y)
            return self.num_samples
    
#     def _generate(self, batch_size=32, num_classes=19*19, count_flag=False):
#         name_list = list(self.name_list)
#         random.shuffle(name_list)
#         for ifile in itertools.islice(name_list, None):
#             file_features = ifile.format('features')
#             file_labels = ifile.format('labels')
#             #print(file_features, file_labels)
#             x = np.array(self.hdf5_file[file_features], dtype='float32')
#             x = np.transpose(x, (0,2,3,1))
#             y = np.array(self.hdf5_file[file_labels])
#             yield x, y
    def _generate(self, batch_size=32, num_classes=19*19, count_flag=False):
        name_list = list(self.name_list)
        if self.data_type == 'test':
            nn = 1
        else:
            nn = self.nn
        for _ in range(nn):
            random.shuffle(name_list)
            for ifile in itertools.islice(name_list, None):
                file_features = ifile.format('features')
                file_labels = ifile.format('labels')
                #print(file_features, file_labels)
                x = np.array(self.hdf5_file[file_features], dtype='float32')
                x = np.transpose(x, (0,2,3,1))
                y = np.array(self.hdf5_file[file_labels])
                yield x, y
    
    def generate(self, batch_size=32, num_classes=19*19):
        while True:
            for x, y in self._generate(batch_size, num_classes):
                #x = x.astype('float32')
                y = to_categorical(y.astype(int), num_classes)
                #print(x, y)
                while x.shape[0] >= batch_size:
                    x_batch, x = x[:batch_size], x[batch_size:]
                    y_batch, y = y[:batch_size], y[batch_size:]
                    yield x_batch, y_batch

In [12]:
%%time
gen_train = Generator(hdf5_train['data'], 'train')
#gen_train.name_list
#print(gen_train)
gen_train.name_list[:10], len(gen_train.name_list), gen_train.num_samples

CPU times: user 1.84 s, sys: 10.7 ms, total: 1.85 s
Wall time: 1.86 s


(['KGS-2009-19-18837-train_{}_33',
  'KGS-2014-19-13029-train_{}_49',
  'KGS-2006-19-10388-train_{}_40',
  'KGS-2005-19-13941-train_{}_12',
  'KGS-2011-19-19099-train_{}_62',
  'KGS-2015-19-8133-train_{}_13',
  'KGS-2008-19-14002-train_{}_54',
  'KGS-2015-19-8133-train_{}_17',
  'KGS-2009-19-18837-train_{}_24',
  'KGS-2014-19-13029-train_{}_9'],
 761,
 194816)

In [13]:
for ee in itertools.islice(gen_train.generate(), 3):
    x, y = ee
    print(x.shape, y.shape)

(32, 19, 19, 1) (32, 361)
(32, 19, 19, 1) (32, 361)
(32, 19, 19, 1) (32, 361)


In [14]:
%%time
gen_test = Generator(hdf5_test['data'], 'test')
print(gen_test)
gen_test.name_list[:10], len(gen_test.name_list), gen_test.num_samples

CPU times: user 181 ms, sys: 3.71 ms, total: 185 ms
Wall time: 193 ms


(['KGS-2005-19-13941-test_{}_0',
  'KGS-2008-19-14002-test_{}_1',
  'KGS-2008-19-14002-test_{}_4',
  'KGS-2011-19-19099-test_{}_6',
  'KGS-2005-19-13941-test_{}_2',
  'KGS-2007-19-11644-test_{}_1',
  'KGS-2009-19-18837-test_{}_1',
  'KGS-2014-19-13029-test_{}_5',
  'KGS-2005-19-13941-test_{}_7',
  'KGS-2004-19-12106-test_{}_2'],
 73,
 18688)

In [15]:
for ee in itertools.islice(gen_test.generate(), 3):
    x, y = ee
    print(x.shape, y.shape)

(32, 19, 19, 1) (32, 361)
(32, 19, 19, 1) (32, 361)
(32, 19, 19, 1) (32, 361)


In [16]:
go_board_rows, go_board_cols = 19, 19
num_classes = go_board_rows * go_board_cols
encoder = PlaneEncoder((go_board_rows, go_board_cols))

In [17]:
%%time
gen_train.num_samples, gen_test.num_samples

CPU times: user 3 µs, sys: 0 ns, total: 3 µs
Wall time: 7.63 µs


(194816, 18688)

In [18]:
'''
small
'''
from keras.layers.core import Dense, Activation, Flatten, Reshape, Lambda, Permute
from keras.layers.convolutional import Conv2D, ZeroPadding2D
from keras import backend as K
# from tensorflow.python.keras.layers.core import Dense, Activation, Flatten, Reshape
# from tensorflow.python.keras.layers.convolutional import Conv2D, ZeroPadding2D

input_shape = (go_board_rows, go_board_cols, encoder.num_planes)
model = Sequential()
#model.add(Permute((2,3,1), input_shape=input_shape))
model.add(Conv2D(48, (7, 7), padding='same', input_shape=input_shape, activation='elu'))
model.add(Conv2D(32, (5, 5), padding='same', activation='elu'))
model.add(Conv2D(32, (5, 5), padding='same', activation='elu'))
model.add(Conv2D(32, (5, 5), padding='same', activation='elu'))
model.add(
        Conv2D(filters=1, kernel_size=1, padding='same',
               activation='elu'))
model.add(Flatten())
model.add(Dense(num_classes, activation='softmax'))

#model.compile(loss='categorical_crossentropy', optimizer='sgd', metrics=['accuracy'])
model.compile(loss='categorical_crossentropy', optimizer='adadelta', metrics=['accuracy'])
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 19, 19, 48)        2400      
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 19, 19, 32)        38432     
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 19, 19, 32)        25632     
_________________________________________________________________
conv2d_4 (Conv2D)            (None, 19, 19, 32)        25632     
_________________________________________________________________
conv2d_5 (Conv2D)            (None, 19, 19, 1)         33        
_________________________________________________________________
flatten_1 (Flatten)          (None, 361)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 361)              

In [19]:
# '''
# large
# '''
# from keras.layers.core import Dense, Activation, Flatten, Reshape
# from keras.layers.convolutional import Conv2D, ZeroPadding2D
# # from tensorflow.python.keras.layers.core import Dense, Activation, Flatten, Reshape
# # from tensorflow.python.keras.layers.convolutional import Conv2D, ZeroPadding2D

# input_shape = (encoder.num_planes, go_board_rows, go_board_cols)
# #network_layers = small.layers(input_shape)
# #network_layers = large.layers(input_shape)
# model = Sequential()
# # for layer in network_layers:
# #     print(layer)
# #     model.add(layer)
# model.add(Reshape((go_board_rows, go_board_cols, encoder.num_planes), input_shape=input_shape))
# model.add(ZeroPadding2D((3, 3)))
# model.add(Conv2D(64, (7, 7), padding='valid'))
# model.add(Activation('relu'))
# model.add(ZeroPadding2D((2, 2)))
# model.add(Conv2D(64, (5, 5)))
# model.add(Activation('relu'))
# model.add(ZeroPadding2D((2, 2)))
# model.add(Conv2D(64, (5, 5)))
# model.add(Activation('relu'))
# model.add(ZeroPadding2D((2, 2)))
# model.add(Conv2D(48, (5, 5)))
# model.add(Activation('relu'))
# model.add(ZeroPadding2D((2, 2)))
# model.add(Conv2D(48, (5, 5)))
# model.add(Activation('relu'))
# model.add(ZeroPadding2D((2, 2)))
# model.add(Conv2D(32, (5, 5)))
# model.add(Activation('relu'))
# model.add(ZeroPadding2D((2, 2)))
# model.add(Conv2D(32, (5, 5)))
# model.add(Activation('relu'))
# model.add(Flatten())
# model.add(Dense(1024))
# model.add(Activation('relu'))
        
# model.add(Dense(num_classes, activation='softmax'))
# model.compile(loss='categorical_crossentropy', optimizer='sgd', metrics=['accuracy'])
# #model.compile(loss='categorical_crossentropy', optimizer='adadelta', metrics=['accuracy'])
# model.summary()

In [20]:
model.inputs, model.outputs

([<tf.Tensor 'conv2d_1_input:0' shape=(None, 19, 19, 1) dtype=float32>],
 [<tf.Tensor 'dense_1/Softmax:0' shape=(None, 361) dtype=float32>])

In [21]:
epochs = 4
batch_size = 32
hst = model.fit_generator(generator=gen_train.generate(batch_size, num_classes),
                    epochs=epochs, verbose=2,
                    steps_per_epoch=gen_train.num_samples / batch_size,
                    validation_data=gen_test.generate(batch_size, num_classes),
                    validation_steps=gen_test.num_samples / batch_size,
                    callbacks=[ModelCheckpoint('checkpoints/model_epoch_{epoch}.h5')])
# hst = model.fit_generator(generator=gen_train.generate(batch_size, num_classes),
#                     epochs=epochs,
#                     steps_per_epoch=5,
#                     validation_data=gen_test.generate(batch_size, num_classes),
#                     validation_steps=5,
#                     callbacks=[ModelCheckpoint('checkpoints/model_epoch_{epoch}.h5')])

Epoch 1/4
 - 46s - loss: 3.5722 - accuracy: 0.3832 - val_loss: 5.1519 - val_accuracy: 0.3162
Epoch 2/4
 - 41s - loss: 3.2714 - accuracy: 0.3932 - val_loss: 4.9412 - val_accuracy: 0.3221
Epoch 3/4
 - 42s - loss: 3.1258 - accuracy: 0.4021 - val_loss: 5.1080 - val_accuracy: 0.3283
Epoch 4/4
 - 41s - loss: 3.0064 - accuracy: 0.4121 - val_loss: 5.1068 - val_accuracy: 0.3352


In [22]:
epochs = 4
batch_size = 64
hst = model.fit_generator(generator=gen_train.generate(batch_size, num_classes),
                    epochs=epochs, verbose=2,
                    steps_per_epoch=gen_train.num_samples / batch_size,
                    validation_data=gen_test.generate(batch_size, num_classes),
                    validation_steps=gen_test.num_samples / batch_size,
                    callbacks=[ModelCheckpoint('checkpoints/model_epoch_{epoch}.h5')])
# hst = model.fit_generator(generator=gen_train.generate(batch_size, num_classes),
#                     epochs=epochs,
#                     steps_per_epoch=5,
#                     validation_data=gen_test.generate(batch_size, num_classes),
#                     validation_steps=5,
#                     callbacks=[ModelCheckpoint('checkpoints/model_epoch_{epoch}.h5')])

Epoch 1/4
 - 24s - loss: 2.8386 - accuracy: 0.4287 - val_loss: 3.9640 - val_accuracy: 0.3444
Epoch 2/4
 - 24s - loss: 2.7012 - accuracy: 0.4455 - val_loss: 5.5618 - val_accuracy: 0.3555
Epoch 3/4
 - 25s - loss: 2.5730 - accuracy: 0.4654 - val_loss: 1.5591e-04 - val_accuracy: 0.3665
Epoch 4/4
 - 24s - loss: 2.4459 - accuracy: 0.4868 - val_loss: 7.9390e-05 - val_accuracy: 0.3769


In [23]:
epochs = 4
batch_size = 128
hst = model.fit_generator(generator=gen_train.generate(batch_size, num_classes),
                    epochs=epochs, verbose=2,
                    steps_per_epoch=gen_train.num_samples / batch_size,
                    validation_data=gen_test.generate(batch_size, num_classes),
                    validation_steps=gen_test.num_samples / batch_size,
                    callbacks=[ModelCheckpoint('checkpoints/model_epoch_{epoch}.h5')])
# hst = model.fit_generator(generator=gen_train.generate(batch_size, num_classes),
#                     epochs=epochs,
#                     steps_per_epoch=5,
#                     validation_data=gen_test.generate(batch_size, num_classes),
#                     validation_steps=5,
#                     callbacks=[ModelCheckpoint('checkpoints/model_epoch_{epoch}.h5')])

Epoch 1/4
 - 15s - loss: 2.2929 - accuracy: 0.5103 - val_loss: 1.8154e-04 - val_accuracy: 0.3875
Epoch 2/4
 - 16s - loss: 2.1932 - accuracy: 0.5258 - val_loss: 5.1389 - val_accuracy: 0.3912
Epoch 3/4
 - 15s - loss: 2.1045 - accuracy: 0.5403 - val_loss: 4.4787 - val_accuracy: 0.3973
Epoch 4/4
 - 15s - loss: 2.0298 - accuracy: 0.5518 - val_loss: 4.6118 - val_accuracy: 0.4005


In [24]:
# from keras.optimizers import Adam
# model.compile(loss='categorical_crossentropy', optimizer=Adam(0.001), metrics=['accuracy'])

In [25]:
# epochs = 5
# batch_size = 32
# hst = model.fit_generator(generator=gen_train.generate(batch_size, num_classes),
#                     epochs=epochs,
#                     steps_per_epoch=gen_train.num_samples / batch_size,
#                     validation_data=gen_test.generate(batch_size, num_classes),
#                     validation_steps=gen_test.num_samples / batch_size,
#                     callbacks=[ModelCheckpoint('checkpoints/model_epoch_{epoch}.h5')])

In [26]:
ls -la checkpoints

total 10632
drwxr-xr-x 2 root root    4096 Jan 21 03:21 ./
drwxr-xr-x 4 root root    4096 Jan 21 03:18 ../
-rw-r--r-- 1 root root 2716876 Jan 21 03:23 model_epoch_1.h5
-rw-r--r-- 1 root root 2716876 Jan 21 03:23 model_epoch_2.h5
-rw-r--r-- 1 root root 2716876 Jan 21 03:24 model_epoch_3.h5
-rw-r--r-- 1 root root 2716876 Jan 21 03:24 model_epoch_4.h5
